In [2]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 불러오기
load_dotenv()

# 환경 변수 읽기
openai_api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 라이브러리가 환경 변수에서 키를 읽을 수 있도록 설정
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

print("API Key configured:", "OPENAI_API_KEY" in os.environ)

from openai import OpenAI
client = OpenAI()

API Key configured: True


In [3]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',  # omni model
    messages = [
        {'role':'user', 'content':'파이썬에 대해 알려줘.'}
    ]
)

In [5]:
response.choices[0].message.content

'파이썬(Python)은 다양한 용도로 사용되는 고급 프로그래밍 언어입니다. 1991년 귀도 반 로섬(Guido van Rossum)이 처음 개발했으며, 현재까지도 많은 개발자들 사이에서 인기가 있습니다. 파이썬에 대한 주요 특징은 다음과 같습니다.\n\n1. **간결하고 읽기 쉬운 문법**: 파이썬은 간결한 문법으로 유명하여, 배우기 쉽고 코드를 읽고 이해하기 쉬운 장점이 있습니다.\n\n2. **다양한 라이브러리와 프레임워크**: 파이썬은 웹 개발(예: Django, Flask), 데이터 분석(예: Pandas, NumPy), 기계 학습(예: TensorFlow, Scikit-learn) 등 여러 분야에 유용한 강력한 라이브러리와 프레임워크를 제공합니다.\n\n3. **플랫폼 독립성**: 파이썬은 여러 운영체제에서 구동될 수 있어, Windows, macOS, Linux 등 다양한 환경에서 사용할 수 있습니다.\n\n4. **객체 지향 프로그래밍**: 파이썬은 객체 지향 프로그래밍을 지원하여, 재사용 가능한 코드 작성이 용이합니다. 그러나 함수형 프로그래밍 스타일도 지원합니다.\n\n5. **커뮤니티와 생태계**: 파이썬에는 대규모의 커뮤니티가 있으며, 다양한 문서와 자원, 튜토리얼이 있어 학습과 문제 해결에 큰 도움이 됩니다.\n\n6. **용도**: 웹 개발, 데이터 과학, 인공지능, 자동화 스크립트, 게임 개발 등 다양한 분야에서 사용됩니다.\n\n파이썬을 배우고 사용하기 좋은 방법으로는 온라인 강의, 책, 공식 문서, 커뮤니티 포럼 등을 추천합니다. 처음 시작할 때는 간단한 프로젝트를 통해 실력을 키우는 것이 좋습니다.'

# 출력 형식 지정

In [4]:
message = {'role':'user', 'content':'오늘 서울 날씨를 json형태로 제공해줘. 키는 weather, temperature로 설정해서.'}

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [message],
    response_format={'type':'json_object'}
)
assistant_reply = response.choices[0].message.content
print('ChatGPT 답변:', assistant_reply)

ChatGPT 답변: {"weather": "맑음", "temperature": "15°C"}


In [5]:
import json
data = json.loads(assistant_reply)
data.keys()

dict_keys(['weather', 'temperature'])

In [6]:
# 서울, 부산 + ... 날씨 정보를 받아서 json으로 추출
message = {'role':'user', 'content':'오늘 서울과 부산, 대전, 대구의 날씨를 json형태로 제공해줘. 키는 weather, temperature로 설정해서.'}

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [message],
    response_format={'type':'json_object'}
)
assistant_reply = response.choices[0].message.content
print('ChatGPT 답변:', assistant_reply)

ChatGPT 답변: {"서울": {"weather": "맑음", "temperature": "18도"}, "부산": {"weather": "흐림", "temperature": "20도"}, "대전": {"weather": "비", "temperature": "17도"}, "대구": {"weather": "맑음", "temperature": "19도"}}


In [8]:
# 서울의 날씨는 현재 ... 이고, 온도는 ... 입니다.
weather_data = json.loads(assistant_reply)

for city, data in weather_data.items():
    print(f'{city}의 날씨는 현재 {data['weather']}이고, 온도는 {data['temperature']} 입니다.')

서울의 날씨는 현재 맑음이고, 온도는 18도 입니다.
부산의 날씨는 현재 흐림이고, 온도는 20도 입니다.
대전의 날씨는 현재 비이고, 온도는 17도 입니다.
대구의 날씨는 현재 맑음이고, 온도는 19도 입니다.


# CoT 프롬프트

In [11]:
question = '철수는 사과 10개를 가지고 있었는데, 그 중 3개를 먹은 후 5개를 더 구입했습니다. 철수가 현재 가지고 있는 사과는 몇 개일까요?'

response = client.chat.completions.create(
    messages=[{'role':'user', 'content':question}],
    model = 'gpt-4o-mini',
    temperature = 0
)

print('CoT적용전:', response.choices[0].message.content)

CoT적용전: 철수가 처음에 가지고 있던 사과는 10개입니다. 그 중 3개를 먹었으므로 남은 사과는 10 - 3 = 7개입니다. 이후 5개를 더 구입했으므로 현재 가지고 있는 사과는 7 + 5 = 12개입니다. 

따라서 철수가 현재 가지고 있는 사과는 12개입니다.


In [12]:
cot_prompt = '(단계를 나눠서 생각해 보자)\n' + question

response = client.chat.completions.create(
    messages=[{'role':'user', 'content':cot_prompt}],
    model = 'gpt-4o-mini',
    temperature = 0
)

print('CoT적용후:', response.choices[0].message.content)

CoT적용후: 단계를 나눠서 생각해 보겠습니다.

1. 철수가 처음 가지고 있던 사과의 개수: 10개
2. 철수가 먹은 사과의 개수: 3개
3. 철수가 구입한 사과의 개수: 5개

이제 계산해 보겠습니다.

1. 철수가 사과를 먹은 후 남은 사과의 개수:
   10개 - 3개 = 7개

2. 철수가 구입한 사과를 더한 후의 개수:
   7개 + 5개 = 12개

따라서, 철수가 현재 가지고 있는 사과는 12개입니다.


In [13]:
question = '어떤 수를 3배한 결과에 4를 더하면 19가 됩니다. 그 수는 무엇일까요? 단계별로 풀어보세요.'

response = client.chat.completions.create(
    messages=[{'role':'user', 'content':question}],
    model = 'gpt-4o-mini',
    temperature = 0
)

print('CoT적용:', response.choices[0].message.content)

CoT적용: 주어진 문제를 단계별로 풀어보겠습니다.

1. **문제 이해하기**: 어떤 수를 \( x \)라고 할 때, 이 수를 3배한 결과에 4를 더하면 19가 된다는 식을 세울 수 있습니다. 이를 수식으로 표현하면 다음과 같습니다.
   \[
   3x + 4 = 19
   \]

2. **식 정리하기**: 이제 이 식을 풀어보겠습니다. 먼저, 양변에서 4를 빼줍니다.
   \[
   3x + 4 - 4 = 19 - 4
   \]
   \[
   3x = 15
   \]

3. **x 구하기**: 이제 양변을 3으로 나누어 \( x \)의 값을 구합니다.
   \[
   x = \frac{15}{3}
   \]
   \[
   x = 5
   \]

4. **결과 확인하기**: 구한 \( x \)의 값이 맞는지 확인해보겠습니다. \( x = 5 \)일 때, 3배한 결과에 4를 더하면:
   \[
   3 \times 5 + 4 = 15 + 4 = 19
   \]
   맞습니다.

따라서, 그 수는 **5**입니다.


# Self consistency 기법

In [ ]:
question = '어떤 수를 3배한 결과에 4를 더하면 19가 됩니다. 그 수는 무엇일까요? 단계별로 풀어보세요.'
message=[{'role':'user', 'content':question}]
answers = []

for i in range(5):
    response = client.chat.completions.create(
        messages = message,
        model = 'gpt-4o-mini',
        temperature = 1.0
)

    answer = response.choices[0].message.content.strip()
    answers.append(answer)

    print(f'응답 {i+1} : {answer}')

응답 1 : 어떤 수를 \( x \)라고 가정하겠습니다. 문제에서, 이 수를 3배한 결과에 4를 더하면 19가 된다고 했으므로, 이를 식으로 표현하면 다음과 같습니다:

\[
3x + 4 = 19
\]

이제 이 방정식을 단계별로 풀어보겠습니다.

1. 양쪽에서 4를 뺍니다:
\[
3x + 4 - 4 = 19 - 4
\]
\[
3x = 15
\]

2. 양쪽을 3으로 나누어 \( x \)의 값을 구합니다:
\[
x = \frac{15}{3}
\]
\[
x = 5
\]

따라서, 그 수는 \( 5 \)입니다.

검산을 해보겠습니다:
- 5를 3배하면 \( 3 \times 5 = 15 \)입니다.
- 여기에 4를 더하면 \( 15 + 4 = 19 \)가 됩니다.

결과가 맞음을 확인하였으므로, 정답은 \( 5 \)입니다.
응답 2 : 문제를 해결하기 위해 단계별로 풀어보겠습니다.

1. **문제 이해**: 어떤 수를 \( x \)라고 했을 때, 이를 3배한 후 4를 더하면 19가 됩니다. 식으로 표현하면 다음과 같아요:
   \[
   3x + 4 = 19
   \]

2. **식 정리**: 이제 이 식에서 \( x \)를 구하기 위해 정리해보겠습니다.

   우선, 양쪽에서 4를 빼줍니다:
   \[
   3x + 4 - 4 = 19 - 4
   \]
   정리하면:
   \[
   3x = 15
   \]

3. **x 구하기**: 이제 3으로 나누어 \( x \)를 구합니다:
   \[
   x = \frac{15}{3}
   \]
   계산하면:
   \[
   x = 5
   \]

4. **결과 확인**: 구한 값이 맞는지 확인해보겠습니다. \( x = 5 \)일 때, \( 3 \cdot 5 + 4 \)의 값을 계산해보면:
   \[
   3 \cdot 5 + 4 = 15 + 4 = 19
   \]
   맞습니다!

따라서, 그 수는 \( \boxed{5} \)입니다.
응답 3 : 어떤 수를 \( x \)로 설정하겠습니다. 문제에서 주어진 조건을 식으로

In [18]:
final_answer = [ans.split()[-1] for ans in answers]
import collections
counter = collections.Counter(final_answer)
counter.most_common(1)
# 정확한 카운팅을 하려면 출력형식 지정이 필요함

[('\\)입니다.', 4)]

# Reflexion (자기 피드백 반복 프롬프트 설계)

In [19]:
question = '7의 2승에 5를 곱한 값은 무엇인가?'

response1 = client.chat.completions.create(
    messages=[{'role':'user', 'content':question}],
    model = 'gpt-3.5-turbo',
    temperature = 0
)
answer = response1.choices[0].message.content.strip() # 모델이 출력한 응답

correct_answer = 245 # 직접 주는 피드백
if str(correct_answer) in answer:
    feedback = '정답!'
    need_retry = False
else:
    feedback = '오답! 힌트는 7의 제곱값을 정확히 구한 다음 곱하기'
    need_retry = True

print(feedback)

정답!


# 틀린답을 출력하는 모델은?

In [20]:
# 2차 답변 생성
if need_retry:
    retry_prompt = f'이전 답변: {answer}, 피드백: {feedback} \n 다시 답 구해.'
    
    response2 = client.chat.completions.create(
        messages=[{'role':'user', 'content':retry_prompt}],
        model = 'gpt-3.5-turbo',
        temperature = 0
    )
    
    answer2 = response2.choices[0].message.content.strip()
    print(answer2)

In [24]:
prompt = '기후 변화의 원인과 해결 방안에 대해 간략한 에세이를 작성해줘.'

response1 = client.chat.completions.create(
    messages=[{'role':'user', 'content':prompt}],
    model = 'gpt-4o-mini',
    temperature = 1.0
)
answer1 = response1.choices[0].message.content
print('1차 답변:', answer1)

feedback_prompt = f'위 답변에 대해 자기 비평을 해. 부족한 부분이나 모호한 부분, 구체성이 떨어지는 부분을 평가해.\n 답변:\n{answer1}'

feedback = client.chat.completions.create(
    messages=[{'role':'user', 'content':feedback_prompt}],
    model = 'gpt-4o-mini',
    temperature = 1.0
)
feedback_self = feedback.choices[0].message.content
print('피드백:', feedback_self)


new_prompt = f'다음은 너가 작성한 답변과 피드백이야. 피드백을 반영하여 답변을 개선해 줘.\n원래 답변:\n{answer1}\n피드백:\n{feedback_self}'

response2 = client.chat.completions.create(
    messages=[{'role':'user', 'content':new_prompt}],
    model = 'gpt-4o-mini',
    temperature = 1.0
)
answer2 = response2.choices[0].message.content
print("2차 답변:", answer2)

1차 답변: ### 기후 변화의 원인과 해결 방안

기후 변화는 현대 사회가 직면한 가장 심각한 환경 문제 중 하나로, 지구의 기온 상승, 해수면 상승, 극단적인 날씨 현상을 초래하고 있습니다. 기후 변화의 주된 원인은 온실가스의 대기 중 농도 증가입니다. 이 온실가스는 주로 인간 활동, 특히 화석 연료의 연소, 산업 공정, 농업 및 삼림 파괴로 인해 발생합니다. 이 과정에서 이산화탄소, 메탄, 아산화질소 등의 가스가 대기에 방출되어 온실 효과를 강화시킵니다.

기후 변화의 해결 방안을 모색하기 위해서는 다음과 같은 방법을 고려해야 합니다. 첫째, 재생 가능 에너지의 확대가 필요합니다. 태양광, 풍력, 수력 등 청정 에너지원으로의 전환은 화석 연료 의존도를 낮추고, 온실가스 배출을 줄이는 데 기여할 수 있습니다. 둘째, 에너지 효율성을 개선해야 합니다. 건물, 교통수단, 산업 공정에서 에너지 낭비를 줄임으로써 전체 에너지 사용량을 감소시킬 수 있습니다. 셋째, 지속 가능한 농업과 삼림 관리가 중요합니다. 지속 가능한 농업 practices를 통해 탄소 배출을 줄이고, 삼림 복원 및 보호를 통해 탄소를 저장하는 시스템을 강화해야 합니다.

마지막으로, 개인의 행동 변화도 중요한 요소입니다. 친환경 소비를 지향하고, 대중교통을 이용하며, 재활용과 쓰레기 줄이기 등을 실천함으로써 개인이 기후 변화 해결에 기여할 수 있습니다. 

결론적으로, 기후 변화는 단순한 환경 문제가 아니라 인간의 생존과 직결된 중대한 사안입니다. 각국 정부, 기업, 개인이 협력하여 기후 변화의 원인을 이해하고, 실천 가능한 해결 방안을 강구하는 것이 필수적입니다. 지속 가능한 미래를 위해서는 지금 행동해야 합니다.
피드백: 위의 답변에 대한 자기 비평을 해보면, 다음과 같은 부족한 부분, 모호한 부분, 그리고 구체성이 떨어지는 요소들이 있음을 알 수 있습니다.

1. **구체성 부족**:
   - 기후 변화에 대한 원인과 해결 방안이 나열되어 있지만, 각 방안에 대한 구체적인 실행 계획이나 

In [21]:
prompt = '기후 변화의 원인과 해결 방안에 대해 15문장 이내의 간략한 에세이를 작성해줘.'
answers = []

for i in range(3):
    response = client.chat.completions.create(
        messages=[{'role':'user', 'content':prompt}],
        model = 'gpt-4o-mini',
        temperature = 1.0
    )
    answer = response.choices[0].message.content
    answers.append(answer)
    
    feedback_prompt = f'위 답변에 대해 자기 비평을 해. 부족한 부분이나 모호한 부분, 구체성이 떨어지는 부분을 평가해.\n 답변:\n{answer}'
    
    feedback = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user", "content": feedback_prompt}],
        temperature = 1.0
    )
    feedback_self = feedback.choices[0].message.content
    
    prompt = f'다음은 너가 작성한 답변과 피드백이야. 피드백을 반영하여 답변을 개선해 줘.\n원래 답변:\n{answer}\n피드백:\n{feedback_self}'
    print(f'\n[{i+1}차 답변]\n', answer)
    print(f'[{i+1}차 피드백]\n', feedback_self)


[1차 답변]
 기후 변화의 주요 원인은 주로 인간의 활동에서 비롯됩니다. 산업화와 도시화는 온실가스를 대량 배출하게 되었고, 이로 인해 지구의 평균 온도가 상승하고 있습니다. 특히 이산화탄소, 메탄, 아산화질소 같은 온실가스는 대기 중에 장기간 남아 생태계를 변화시킵니다. 이러한 기후 변화는 극단적인 날씨, 해수면 상승, 생물 다양성 감소 등의 문제를 초래하고 있습니다.

해결 방안으로는 우선 재생 가능한 에너지원의 사용을 확대해야 합니다. 태양광, 풍력, 수력 등 청정 에너지를 채택하면 화석연료 의존도를 줄일 수 있습니다. 또한, 에너지 효율성을 높이는 기술 개발과 전기차 및 대중교통 이용을 장려하는 정책도 필요합니다. 신재생 에너지를 통한 전환뿐만 아니라, 산림 보호와 복원 또한 중요합니다. 나무는 탄소를 흡수하고 생태계를 유지하는 데 필수적입니다. 마지막으로, 개인과 사회가 기후 변화의 심각성을 인식하고, 지속 가능한 생활 방식을 실천하는 것이 중요합니다. 

모두의 협력이 필요한 이 문제는 단지 환경을 넘어 경제, 사회, 그리고 우리의 미래에 직접적인 영향을 미치므로, 즉각적인 행동이 요구됩니다.
[1차 피드백]
 위 답변은 기후 변화의 원인과 해결 방안에 대해 전반적으로 잘 설명하고 있으나, 몇 가지 부족한 점과 개선할 부분이 있습니다.

1. **구체성 부족**: 기후 변화의 원인에 대해 언급했지만, 각 온실가스의 배출원이나 특정 산업의 영향을 구체적으로 설명하지 않았습니다. 예를 들어, 이산화탄소는 주로 화석 연료의 연소에서 발생하는데, 어떤 산업이 그 비중을 차지하는지 명시하면 독자가 이해하는 데 도움이 될 것입니다.

2. **모호한 용어 사용**: "산업화와 도시화"라는 표현은 일반적이어서 이를 더 구체적인 예시로 보강할 필요가 있습니다. 예를 들어, 농업, 제조업, 건설업 등이 어떻게 온실가스를 배출하는지에 대한 사례를 제시하면 좋겠습니다.

3. **해결 방안의 깊이 부족**: 재생 가능한 에너지원의 사용 확대에 대한 언급은 좋지만, 